In [1]:
ls

sample_data/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip uninstall torch

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.9.0+cu102.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.9.0+cu102


In [5]:
!pip install torch==1.6.0
!pip install torchvision==0.7.0
!pip install coremltools

     |████████████████████████████████| 748.8 MB 18 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
     |████████████████████████████████| 5.9 MB 4.1 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
     |████████████████████████████████| 3.4 MB 4.1 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=ec19304b3a8ecd55cfae9afd28153a49a14eca2820981113f5018841952b982d
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [ ]:
!pip3 install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 154.6 MB 39 kB/s 
     |████████████████████████████████| 5.1 MB 3.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0
    Uninstalling torchvision-0.7.0:
      Successfully uninstalled torchvision-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0+cpu which is incompatible.


In [ ]:
!pip install coremltools

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=324625ef89f34c047c3ed8acb1808d2607287122627b1dd115b1d212afb20ff7
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [ ]:
!pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.


In [10]:
import torch

from torchvision import transforms
from PIL import Image
import numpy as np

import coremltools as ct

%matplotlib inline
from matplotlib.pyplot import imshow

model = torch.load('./drive/MyDrive/beverage_epoch250.pt').cpu().eval()

input_image = Image.open('./drive/MyDrive/봉봉1.jpg')
# imshow(np.asarray(input_image))

preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            ),
        ])

input_tensor = preprocess(input_image)
image_batch = input_tensor.unsqueeze(0)

with torch.no_grad():
    output = model(image_batch)
    _, preds = torch.max(output, 1)

traced_model = torch.jit.trace(model, image_batch).cpu()

class_labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

# Convert to Core ML using the Unified Conversion API
mlmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="Image", shape=image_batch.shape)],
	classifier_config = ct.ClassifierConfig(class_labels)
)

# Save model
mlmodel.save("beverage.mlmodel")

Translating MIL ==> MLModel Ops: 100%|██████████| 268/268 [00:01<00:00, 159.59 ops/s] 


In [11]:
mv beverage.mlmodel drive/MyDrive